# L'affidabilità tra giudici {#sec-interrater-reliability}

**Prerequisiti**

**Concetti e Competenze Chiave**

**Preparazione del Notebook**

In [ ]:
here::here("code", "_common.R") |>
    source()

# Load packages
if (!requireNamespace("pacman")) install.packages("pacman")
pacman::p_load(lme4)

## Introduzione

Se la valutazione di un test dipende da giudizi soggettivi, è essenziale valutare il grado di accordo tra diversi individui che assegnano i punteggi. Questo tipo di affidabilità è conosciuto come affidabilità inter-valutatore (o inter-scorer, inter-rater reliability in inglese). L'affidabilità inter-valutatore si riferisce alla coerenza o concordanza nelle valutazioni tra due o più giudici che esaminano lo stesso fenomeno o campione. Questo aspetto è fondamentale, in quanto indica il livello di similitudine nei risultati ottenuti da diversi valutatori quando si trovano a esprimere giudizi o a effettuare misurazioni in contesti simili. 

In pratica, l'affidabilità inter-valutatore si misura calcolando quanto siano vicini tra loro i punteggi o le valutazioni fornite da diversi osservatori. Una forte affidabilità inter-valutatore indica che il test o lo strumento di valutazione produce risultati consistenti e riproducibili, nonostante la presenza di diversi valutatori. Questo è cruciale per garantire che le conclusioni tratte dai risultati del test siano valide e affidabili.

L'affidabilità tra giudici è un aspetto fondamentale in molti ambiti della psicologia. Consideriamo il caso di una diagnosi psicologica. Quando diversi psicologi valutano i sintomi di un paziente, è essenziale che le loro valutazioni siano coerenti per garantire una diagnosi accurata. Ad esempio, nella valutazione dei disturbi dell'umore, gli psicologi devono giungere a conclusioni simili basandosi su sintomi osservabili e risposte del paziente a questionari standardizzati. Un esempio nella ricerca potrebbe essere uno studio sull'efficacia di una nuova terapia per l'ansia. Qui, vari terapeuti valutano il livello di ansia dei partecipanti prima e dopo il trattamento. L'affidabilità tra le valutazioni di questi terapeuti è cruciale per validare l'efficacia del trattamento.

Questo capitolo esamina l'affidabilità tra giudici in due contesti: con due giudici e con giudici molteplici. Nel primo caso, analizzeremo l'accordo nominale e il Kappa di Cohen. Nel secondo caso, useremo l'ICC, l'ANOVA ad una via e l'ANOVA a due vie. In questo contesto, i modelli a effetti misti si dimostrano fondamentali per gestire disegni con giudici molteplici, catturando le variazioni tra i giudici e tra i partecipanti e fornendo una visione completa dell'affidabilità delle valutazioni.

Nella discussione seguente useremo i dati dello studio [The Reliability and Validity of Speech-Language Pathologists’ Estimations of Intelligibility in Dysarthria](https://www.mdpi.com/2076-3425/12/8/1011/) di @hirsch2022reliability. In questo studio, i giudici valutano la "Percentuale di Intelligibilità del Discorso" di persone affette da disartria. Questa misura soggettiva descrive la percentuale di discorso compreso durante una conversazione con un parlante che soffre di disartria, una condizione che influisce sulla chiarezza e sull'intelligibilità della parola. I giudici ascoltano e valutano le registrazioni vocali dei partecipanti affetti da disartria e stimano la percentuale di parole o frasi che sono in grado di comprendere chiaramente. Questa valutazione fornisce una misura importante dell'impatto della disartria sulla comunicazione verbale quotidiana.

In [4]:
slp_dat <- read.csv("https://osf.io/download/p9gqk/")
slp_vas_wide <- slp_dat |>
    select(slpID, Speaker, slp_VAS) |>
    pivot_wider(names_from = slpID, values_from = slp_VAS)
head(slp_vas_wide)

Speaker,slp10,slp11,slp13,slp14,slp15,slp16,slp17,slp18,slp19,...,slp21,slp22,slp23,slp25,slp3,slp4,slp5,slp6,slp8,slp9
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AF1,96.28,100.00,96.56,99.30,100.00,12.58,100.00,88.16,52.58,...,77.42,73.64,100.00,100.00,37.82,73.24,55.01,100.00,53.94,85.07
AF9,12.32,34.68,0.86,3.52,12.58,0.00,6.69,0.00,7.42,...,13.55,0.00,0.00,2.90,0.00,14.51,1.72,NA,6.76,10.28
ALSF6,NA,30.84,5.73,73.94,73.87,4.52,45.13,41.92,26.13,...,25.81,8.31,18.46,86.77,0.00,26.20,3.72,NA,9.86,37.75
ALSF7,93.98,96.31,61.32,64.37,100.00,7.74,69.92,88.16,37.42,...,59.35,34.10,90.63,53.23,99.71,63.10,56.73,88.83,64.93,53.80
ALSF9,29.51,52.20,43.55,46.76,60.65,23.87,25.21,91.64,37.10,...,57.74,29.51,50.14,64.84,0.00,31.13,40.11,16.91,5.92,50.00
ALSM1,96.28,NA,29.23,69.44,99.35,82.26,70.19,79.72,78.39,...,83.87,84.24,92.29,100.00,100.00,18.03,58.74,66.48,51.83,60.28


## Due giudici

Selezioniamo unicamente due giudici.

In [5]:
slp_2rater <- slp_vas_wide |>
    select(slp14, slp15)
head(slp_2rater)

slp14,slp15
<dbl>,<dbl>
99.30,100.00
3.52,12.58
73.94,73.87
64.37,100.00
46.76,60.65
69.44,99.35


### Accordo nominale

Per calcolare l'accordo nominale, è necessario analizzare quali valutazioni tra i due giudici corrispondono in modo perfettamente uguale. Utilizzando questa definizione, si ottiene:

In [6]:
with(slp_2rater, mean(slp14 == slp15))

[1] 0.1

Se invece modifichiamo leggermente la definizione e affermiamo che l'accordo è raggiunto quando rimane invariato anche dopo l'arrotondamento, si ottiene:

In [7]:
slp_2round <- round(slp_2rater / 10)
slp_2round |> print()

   slp14 slp15
1     10    10
2      0     1
3      7     7
4      6    10
5      5     6
6      7    10
7      8     9
8      8     9
9      9     8
10    10    10
11    10     7
12    10    10
13     6     5
14    10    10
15     6    10
16    10    10
17    10    10
18     9    10
19     9    10
20    10    10


In [8]:
slp_2round <- lapply(slp_2round, FUN = factor, levels = 0:10)
# Contingency table
table(slp_2round)

     slp15
slp14 0 1 2 3 4 5 6 7 8 9 10
   0  0 1 0 0 0 0 0 0 0 0  0
   1  0 0 0 0 0 0 0 0 0 0  0
   2  0 0 0 0 0 0 0 0 0 0  0
   3  0 0 0 0 0 0 0 0 0 0  0
   4  0 0 0 0 0 0 0 0 0 0  0
   5  0 0 0 0 0 0 1 0 0 0  0
   6  0 0 0 0 0 1 0 0 0 0  2
   7  0 0 0 0 0 0 0 1 0 0  1
   8  0 0 0 0 0 0 0 0 0 2  0
   9  0 0 0 0 0 0 0 0 1 0  2
   10 0 0 0 0 0 0 0 1 0 0  7

Il codice precedente fa uso di due funzioni principali: `lapply()` e `table()`. Esso opera su una lista o su un dataframe chiamato `slp_2round`, applicando una trasformazione ai dati e successivamente creando una tabella di contingenza. 

1. `slp_2round <- lapply(slp_2round, FUN = factor, levels = 0:10)`

- `lapply()`: questa funzione appartiene alla famiglia delle funzioni "*apply" in R, che sono utilizzate per applicare una funzione a ciascun elemento di una lista o di un vettore. `lapply()` restituisce sempre una lista come risultato, mantenendo la lunghezza dell'input originale.
  
- `slp_2round`: è la lista o il dataframe su cui si sta operando. Il codice modifica questa variabile, sostituendola con il risultato dell'applicazione di `lapply()`.
  
- `FUN = factor`: specifica che la funzione da applicare a ciascun elemento di `slp_2round` è `factor()`. La funzione `factor()` è utilizzata per codificare un vettore come un fattore, che è utile in R per rappresentare dati categorici.
  
- `levels = 0:10`: indica i livelli del fattore, che in questo caso sono i numeri da 0 a 10. Questo parametro assicura che i fattori creati abbiano esattamente questi livelli, anche se alcuni di essi non appaiono nei dati. Questo è particolarmente utile per garantire la coerenza dei livelli dei fattori attraverso vari elementi di `slp_2round`, soprattutto se si intende confrontarli o combinarli in seguito.

2. `table(slp_2round)`

- `table()`: questa funzione crea una tabella di contingenza dai dati forniti. Una tabella di contingenza è un tipo di tabella in formato matriciale che conta la frequenza di combinazioni specifiche di valori all'interno di una o più variabili categoriche.

Dopo l'applicazione di `lapply()`, `slp_2round` diventa una lista di elementi, ciascuno dei quali è un fattore con livelli da 0 a 10. Quando si passa questa lista modificata alla funzione `table()`, viene generata una tabella di contingenza. Questa tabella riassume le frequenze dei vari livelli dei fattori attraverso gli elementi della lista `slp_2round`.

In [9]:
p0 <- with(slp_2round, mean(slp14 == slp15))
p0

[1] 0.4

Qui, l'arrotondamento serve a ridurre la specificità delle valutazioni, permettendo un confronto più ampio e potenzialmente più flessibile tra le valutazioni dei due giudici​​. Questo approccio riflette una visione più tollerante dell'accordo, riconoscendo che piccole discrepanze nelle valutazioni potrebbero non essere significative in un contesto pratico.

Nel contesto specifico descritto, l'arrotondamento implica dividere le valutazioni per 10 e poi arrotondarle al numero intero più vicino. Questo processo rende i valori leggermente diversi equivalenti. Ad esempio, se un giudice assegna una valutazione di 75 e l'altro assegna 78, dividendo per 10 otteniamo 7.5 e 7.8; arrotondando, entrambi i valori diventano 8. Così, valutazioni originalmente diverse vengono arrotondate allo stesso numero intero, permettendo un accordo più ampio tra i giudici. Questo metodo riconosce l'accordo anche in presenza di piccole variazioni nelle valutazioni, riflettendo una visione più flessibile dell'accordo tra i giudici.

### Kappa di Cohen

L'indice di concordanza Kappa di Cohen è definito nel modo seguente:

$$
\kappa = \frac{p_o - p_c}{1 - p_c},
$$

dove:

- $\kappa$ rappresenta il coefficiente di Kappa di Cohen.
- $p_o$ è l'indice di concordanza osservato tra gli osservatori.
- $p_c$ è l'indice di concordanza attesa per caso.

$p_o$ (indice di concordanza osservato tra gli osservatori):

$$
p_o = \frac{{\text{Numero di accordi osservati}}}{{\text{Numero totale di confronti}}}
$$

In altre parole, $p_o$ è il rapporto tra il numero di volte in cui gli osservatori sono concordi (hanno dato la stessa valutazione) e il numero totale di confronti effettuati.

$p_c$ è l'indice di concordanza attesa per caso:

$$
p_c = \frac{{\sum (\text{Riga i-esima del totale}) \times (\text{Colonna i-esima del totale})}}{{\text{Numero totale di confronti}}^2} 
$$

In altre parole, $p_c$ è il rapporto atteso di concordanza tra gli osservatori nel caso in cui le loro valutazioni siano indipendenti.

Queste formule vengono utilizzate per calcolare il coefficiente di Kappa di Cohen, che misura la concordanza tra osservatori che stanno misurando una variabile categoriale, tenendo conto della concordanza casuale attesa. Un valore di $\kappa$ vicino a 1 indica un'alta concordanza, mentre un valore vicino a 0 indica una concordanza casuale e un valore negativo indica una concordanza peggiore di quella casuale.

In [10]:
slp_2tab <- table(slp_2round)
pc <- sum(colSums(slp_2tab) * rowSums(slp_2tab)) / sum(slp_2tab)^2
(kappa <- (p0 - pc) / (1 - pc))

[1] 0.1666667

Svantaggio: Il coefficiente di Kappa tende a essere basso quando i punteggi sono distribuiti in maniera disomogenea (ad esempio, quando la maggior parte dei punteggi appartiene a determinate categorie). Questa situazione corrisponde esattamente al caso dell'esempio presente.

## Giudici multipli

### Coefficiente $\alpha$

Il coefficiente $\alpha$ di Cronbach è comunemente utilizzato per misurare l'affidabilità interna di un test, cioè la coerenza con cui gli item all'interno del test misurano un costrutto comune. Tuttavia, può essere utilizzato anche per valutare l'affidabilità tra giudici in uno studio in cui più valutatori assegnano punteggi o giudizi a una serie di item.

Nel contesto dell'affidabilità tra giudici, il coefficiente $\alpha$ di Cronbach misura quanto consistentemente i diversi giudici valutano gli stessi item. Un valore di $\alpha$ vicino a 1 indica un alto livello di consistenza (o affidabilità) tra i giudici, suggerendo che stanno valutando gli item in modo simile. Un valore più basso indica una minore coerenza nelle valutazioni tra i giudici.

Per calcolare il coefficiente $\alpha$ di Cronbach in questo contesto, si considerano i punteggi assegnati da ciascun giudice a ciascun item come se fossero item individuali di un test. Quindi, si applica la formula standard del coefficiente $\alpha$ per valutare la coerenza interna di questi "item" (in questo caso, le valutazioni dei giudici).

In [11]:
psych::alpha(slp_vas_wide[-1])

Number of categories should be increased  in order to count frequencies. 



Warning message in cor.smooth(r):
"Matrix was not positive definite, smoothing was done"
In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs < 0 were set to .0

In smc, smcs < 0 were set to .0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs > 1 were set to 1.0

In smc, smcs < 0 were set to .0

In smc, smcs > 1 were set to 1.0




Reliability analysis   
Call: psych::alpha(x = slp_vas_wide[-1])

  raw_alpha std.alpha G6(smc) average_r S/N    ase mean sd median_r
      0.98      0.98       1      0.69  46 0.0076   62 25      0.7

    95% confidence boundaries 
         lower alpha upper
Feldt     0.96  0.98  0.99
Duhachek  0.96  0.98  0.99

 Reliability if an item is dropped:
      raw_alpha std.alpha G6(smc) average_r S/N var.r med.r
slp10      0.97      0.98    0.99      0.68  43 0.014  0.70
slp11      0.97      0.98    0.99      0.68  43 0.014  0.70
slp13      0.97      0.98    0.99      0.69  44 0.015  0.70
slp14      0.97      0.98    0.99      0.69  44 0.015  0.70
slp15      0.97      0.98    0.99      0.69  44 0.014  0.70
slp16      0.98      0.98    0.99      0.70  46 0.014  0.71
slp17      0.97      0.98    0.99      0.68  43 0.015  0.69
slp18      0.97      0.98    0.99      0.69  45 0.013  0.70
slp19      0.97      0.98    0.99      0.69  45 0.014  0.71
slp2       0.97      0.98    1.00      0.69  45 

Nell'esempio in discussione, il valore del coefficiente $\alpha$ di Cronbach è 0.98. Questo indica un'altissima affidabilità interna, suggerendo che i giudici coinvolti nello studio hanno fornito delle valutazioni molto consistenti tra loro. Un valore così elevato di $\alpha$ è raro e suggerisce che ci sia un'alta coerenza nelle valutazioni tra i giudici. In termini pratici, questo significa che si può avere un'elevata fiducia nel fatto che le valutazioni fornite dai diversi giudici siano intercambiabili e riflettano in modo affidabile la "Percentuale di Intelligibilità del Discorso" che viene valutata.

### Coefficiente di correlazione intraclasse

Il Coefficiente di Correlazione Intraclasse (ICC) quantifica il grado di somiglianza tra le unità all'interno dello stesso gruppo. A differenza della maggior parte delle altre misure di correlazione, l'ICC viene impiegato con dati organizzati in gruppi anziché con coppie di osservazioni. Questo lo rende particolarmente idoneo per valutare la concordanza tra giudici che stanno valutando lo stesso insieme di individui o item.

L'ICC si basa sul framework del modello a effetti misti. Questi modelli statistici consentono di analizzare le variazioni dei punteggi sia all'interno dei gruppi (ovvero le differenze tra le osservazioni all'interno dello stesso gruppo) che tra i gruppi (ovvero le differenze tra i gruppi stessi). Pertanto, l'ICC tiene conto di due fonti di varianza: la varianza all'interno dei gruppi (varianza delle valutazioni dei giudici all'interno dello stesso gruppo) e la varianza tra i gruppi (varianza delle valutazioni dei giudici tra gruppi diversi). Inoltre, viene considerato l'errore casuale presente nelle valutazioni.

In sintesi, l'ICC valuta la consistenza delle misurazioni quando queste vengono effettuate da diversi osservatori su un insieme di soggetti o item. Un valore elevato di ICC segnala una forte concordanza tra le valutazioni degli osservatori, indicando che le misurazioni sono affidabili e riproducibili. Al contrario, un ICC basso riflette una discrepanza significativa tra gli osservatori, suggerendo che le valutazioni sono meno consistenti.

Il calcolo dell'ICC si basa sull'uso di un modello statistico ad effetti misti, in cui gli osservatori (giudici) fungono da variabili indipendenti e i punteggi assegnati rappresentano la variabile dipendente. Questo modello consente di esaminare e quantificare diverse fonti di variazione nei dati, tra cui:

1. La variazione attribuibile ai soggetti (o item) valutati (la varianza di interesse principale).
2. La variazione attribuibile agli osservatori (giudici).
3. La variazione dovuta all'interazione tra i soggetti (o item) valutati e gli osservatori.
4. La variazione causata da errore casuale.

L'ICC si calcola dividendo la varianza relativa ai soggetti valutati (o item) per la somma di questa varianza più la varianza dovuta all'errore casuale. Questo rapporto esprime la proporzione della varianza totale dei punteggi che può essere attribuita alla differenziazione tra i soggetti valutati (o item), offrendo un indicatore diretto dell'affidabilità delle misurazioni. Pertanto, un ICC elevato indica che la maggior parte della varianza nei punteggi deriva dalle differenze tra i soggetti valutati (o item), confermando l'affidabilità delle valutazioni fornite dagli osservatori.

Tuttavia, ciò che viene considerato come errore dipende da diverse considerazioni, tra cui:

1. Se il disegno è "crossed" (cioè tutti i giudici valutano ciascun individuo o item) o "nested" (diversi gruppi di giudici valutano ciascun individuo o item). Nel primo caso, l'errore sarà associato alle differenze tra i giudici all'interno dello stesso gruppo di individui (o item) valutati, mentre nel secondo caso l'errore sarà relativo alle differenze tra i gruppi di giudici che valutano individui (o item) diversi.
2. Se i giudici sono considerati "fixed" (non generalizzabili) o "random" (generalizzabili a una "popolazione di giudici"). Nel caso in cui i giudici siano considerati "fixed", si presume che i giudici specifici coinvolti nello studio siano l'unica popolazione rilevante, e l'interesse è focalizzato sulla concordanza tra di loro. Nel caso in cui i giudici siano considerati "random", si ammette che i giudici coinvolti nello studio siano solo un campione di una popolazione più ampia di giudici, e l'interesse è sulla concordanza che può essere generalizzata a tale popolazione più vasta.
3. La distinzione tra "consistenza" (decisioni relative; viene giudicato di interesse solo l'ordinamento dei punteggi) e "accordo assoluto" (decisioni assolute; viene giudicato di interesse il valore effettivo del punteggio). Nel contesto della valutazione dei giudici, la consistenza si riferisce al grado in cui i giudici sono concordi nell'ordine di classificazione degli individui valutati, indipendentemente dai valori numerici dei punteggi. D'altra parte, l'accordo assoluto misura il grado di concordanza tra i giudici riguardo ai valori numerici dei punteggi assegnati agli individui valutati.

Queste considerazioni sono fondamentali quando si analizzano e interpretano i risultati dell'ICC o di altri metodi di valutazione dell'affidabilità tra giudici o osservatori. La scelta del modello statistico e la definizione dell'errore dipendono da queste varie considerazioni, e la comprensione di questi aspetti è essenziale per una valutazione accurata della concordanza e dell'affidabilità nelle misurazioni effettuate da più giudici o osservatori.

### ANOVA ad una via per disegni nidificati

Consideriamo il caso in cui i dati rappresentano da due valutazioni assegnate a ciascuna persona da giudici diversi.

In [12]:
slp_vas_nested <- slp_dat |>
    mutate(SpeakerID = as.numeric(as.factor(Speaker))) |>
    # Select only 10 speakers
    filter(SpeakerID <= 10) |>
    group_by(Speaker) |>
    # Filter specific raters
    filter(row_number() %in% (SpeakerID[1] * 2 - (1:0)))

head(slp_vas_nested)

slpID,Speaker,slp_VAS,slp_VAS.Rel,slp_EST,slp_EST.Rel,SpeakerID
<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>
slp10,AF1,96.28,NA,90,NA,1
slp11,AF1,100.00,NA,80,NA,1
slp13,AF9,0.86,NA,10,NA,2
slp14,AF9,3.52,NA,10,NA,2
slp15,ALSF6,73.87,NA,10,NA,3
slp16,ALSF6,4.52,9.03,5,NA,3


Verifichiamo che ogni giudice abbia fornito due giudizi per soggetto:

In [13]:
slp_vas_nested %>%
  group_by(Speaker) %>%
  summarise(Count = n())

Speaker,Count
<chr>,<int>
AF1,2
AF9,2
ALSF6,2
ALSF7,2
ALSF9,2
ALSM1,2
ALSM4,2
AM1,2
AM3,2


Ci sono 20 giudici:

In [16]:
length(slp_vas_nested$Speaker)

[1] 20

In questo studio, analizziamo dati costituiti da due valutazioni fornite a ciascun individuo da giudici diversi. Il nostro obiettivo è identificare e separare due principali fonti di variazione: le differenze tra i giudici e le variazioni casuali all'interno delle valutazioni di ciascun giudice. 

La sfida risiede nel distinguere queste fonti di variazione, essenziale per determinare la proporzione di variabilità nei punteggi attribuibile a reali differenze tra i giudici, rispetto a quella derivante da errori casuali. Per affrontare questo problema, applichiamo un'analisi della varianza a effetti casuali (Random-Effects ANOVA), utilizzando il modello lineare misto implementato attraverso la funzione `lmer()` di R. Questa metodologia ci permette di stimare separatamente la varianza dovuta alle differenze tra i giudici e quella associata all'errore casuale.

Implementiamo il modello `lmer(slp_VAS ~ 1 + (1 | Speaker), data = slp_vas_nested)`, dove il termine `Speaker` rappresenta i giudici. Attraverso questo modello, trattiamo le intercette associate a ciascun giudice come effetti casuali, consentendoci di catturare le specifiche variazioni tra i giudici al di là delle fluttuazioni casuali.

In [15]:
m1 <- lmer(slp_VAS ~ 1 + (1 | Speaker), data = slp_vas_nested)
summary(m1)

Linear mixed model fit by REML ['lmerMod']
Formula: slp_VAS ~ 1 + (1 | Speaker)
   Data: slp_vas_nested

REML criterion at convergence: 179.6

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.50049 -0.77609 -0.05948  0.87584  1.35589 

Random effects:
 Groups   Name        Variance Std.Dev.
 Speaker  (Intercept) 308.9    17.58   
 Residual             816.8    28.58   
Number of obs: 19, groups:  Speaker, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   53.613      8.627   6.214

Dall'analisi emergono due componenti principali di varianza:
- La varianza attribuibile a differenze intrinseche tra i soggetti valutati (`slpID`), che riflette la variabilità naturale tra gli individui.
- La varianza attribuibile ai giudici (`Speaker`), che misura l'estensione del bias sistematico, ossia la tendenza di alcuni giudici a fornire valutazioni sistematicamente più alte o più basse rispetto ad altri.

In aggiunta, abbiamo la varianza residua, che include l'errore di misurazione e altre fonti di variabilità non spiegate dal modello.

In [17]:
vc_m1 <- as.data.frame(VarCorr(m1))
vc_m1

grp,var1,var2,vcov,sdcor
<chr>,<chr>,<chr>,<dbl>,<dbl>
Speaker,(Intercept),NA,308.9062,17.57573
Residual,NA,NA,816.8141,28.57996


Per valutare l'affidabilità delle valutazioni, calcoliamo l'Intraclass Correlation Coefficient (ICC) utilizzando i dati estratti dal modello. Questo coefficiente quantifica la proporzione della varianza totale attribuibile alle differenze tra i soggetti valutati, offrendoci un indice dell'affidabilità delle valutazioni in contesti dove sono coinvolti giudizi o misurazioni ripetute.

Un ICC prossimo a 1 indica che la maggior parte della variabilità nei dati è dovuta a differenze reali tra i soggetti valutati, mentre un valore più basso suggerisce un'influenza significativa di variazioni casuali o di bias dei giudici sulla variabilità totale osservata.

Calcoliamo l'ICC per una singola valutazione:

$$
ICC = \frac{\sigma^2_{\text{giudici}}} {\sigma^2_{\text{giudici}} + \sigma^2_E}.
$$

Nella formula, l'ICC misura la proporzione della varianza totale del punteggio `slp_VAS` che è attribuibile alla variazione tra i gruppi dei giudici (Speaker) rispetto alla variazione residua o errore.

In [15]:
vc_m1$vcov[1] / (vc_m1$vcov[1] + vc_m1$vcov[2])

[1] 0.2744076

Utilizziamo la funzione `icc()` del pacchetto `performance`:

In [16]:
performance::icc(m1)

ICC_adjusted,ICC_conditional,ICC_unadjusted
<dbl>,<dbl>,<dbl>
0.2744076,0.2744076,0.2744076


Questa formula calcola l'affidabilità per una singola valutazione da un singolo giudice. La formula considera la varianza tra i soggetti rispetto alla varianza totale, che include sia la varianza tra i soggetti sia l'errore di misurazione.

Calcoliamo ora l'ICC per la valutazione media:

$$
ICC = \frac{\sigma^2_{\text{giudici}}} {\sigma^2_{\text{giudici}} + \sigma^2_E / k},
$$

dove $k$ rappresenta il numero di giudizi assegnati a ciascun soggetto da ciascun giudice. 

La formula calcola l'ICC basandosi sulla varianza tra i gruppi (in questo caso, la varianza attribuibile ai giudici, $\sigma^2_{\text{giudici}}$) e la varianza entro i gruppi (l'errore casuale, $\sigma^2_E$), aggiustando per il numero di giudizi ($k$) dati per ogni soggetto. L'inserimento di $k$ nel denominatore serve a normalizzare l'effetto dell'errore casuale in base al numero di giudizi, assumendo che più giudizi per soggetto possano ridurre l'impatto dell'errore casuale sulla stima dell'affidabilità.

Quindi, se l'ICC è vicino a 1, ciò indica che la maggior parte della varianza nei dati è dovuta a differenze reali tra i soggetti valutati, piuttosto che a variazioni casuali o a differenze tra i giudici. In questo modo, l'ICC fornisce un indice utile per valutare l'affidabilità delle valutazioni in studi in cui sono coinvolti giudizi o misurazioni ripetute.

Nel caso presente, ogni giudice fornisce due valutazioni per ogni soggetto:

In [17]:
vc_m1$vcov[1] / (vc_m1$vcov[1] + vc_m1$vcov[2] / 2)

[1] 0.4306434

Questa seconda formula calcola l'affidabilità per la media delle valutazioni di più giudici. È pertinente quando si ha a che fare con più valutazioni per ciascun soggetto e si vuole sapere l'affidabilità della media di queste valutazioni. Questa formula differisce dalla prima perché considera la riduzione dell'errore di misurazione che si verifica quando si calcola la media di più valutazioni.

### ANOVA a due vie e l'Impatto del Bias dei Giudici

L'ANOVA a due vie è uno strumento statistico cruciale per esaminare gli effetti di più fattori, inclusi i giudici, sulla variabile dipendente in contesti di valutazione. Per interpretare accuratamente i risultati, è fondamentale distinguere tra il bias sistematico introdotto dai giudici e la variabilità intrinseca nelle valutazioni.

- **Bias dei Giudici**: Questo si verifica quando i giudici hanno una tendenza costante a fornire valutazioni più alte o più basse rispetto agli altri, influenzando potenzialmente l'equità delle valutazioni.
  
- **Variabilità Intrinseca**: Si riferisce alle differenze naturali nelle valutazioni che emergono dalle percezioni individuali o dalle interpretazioni soggettive dei giudici.

In termini di **Contesto di Valutazione**:
- Nelle analisi di **Consistenza**, dove l'attenzione è sull'ordine relativo delle valutazioni, il bias dei giudici non è considerato problematico poiché non altera questo ordine.
- Nelle analisi di **Accordo**, che si concentrano sulla concordanza dei valori assoluti delle misurazioni, il bias dei giudici è trattato come una fonte di errore significativa.

L'applicazione dell'**ANOVA a due vie** consente di distinguere l'effetto del bias dei giudici rispetto ad altre fonti di varianza, offrendo così un'importante metodologia per valutare l'affidabilità delle valutazioni e l'impatto del bias in diversi contesti.

Utilizzando `lmer(slp_VAS ~ 1 + (1 | Speaker) + (1 | slpID), data = slp_dat)`, analizziamo i dati considerando due principali fonti di variazione:
- **Effetti Casuali dei Giudici (`(1 | Speaker)`):** Valutiamo il bias dei giudici come fonte di variazione, distinguendo tra analisi di consistenza e di accordo a seconda di come questo bias influisce sui risultati.
- **Variabilità Intrinseca tra i Soggetti (`(1 | slpID)`):** Esploriamo le differenze naturali tra i soggetti valutati, fondamentali per comprendere la diversità delle risposte individuali.

Questo approccio ci permette di facilitare la distinzione tra l'effetto del bias dei giudici e altre fonti di varianza, fornendo una metodologia preziosa per valutare l'affidabilità delle valutazioni e l'impatto del bias in diversi contesti.

Eseguiamo l'analisi con `lmer()`:

In [18]:
m2 <- lmer(slp_VAS ~ 1 + (1 | Speaker) + (1 | slpID), data = slp_dat)

Questa formula considera due principali fonti di variazione:

- Effetti Casuali dei Giudici (`(1 | Speaker)`): Valutiamo il bias dei giudici come una fonte di variazione, differenziando tra le analisi di consistenza e di accordo a seconda dell'impatto di questo bias sui risultati.
- Variabilità Intrinseca tra i Soggetti (`(1 | slpID)`): Esploriamo le differenze naturali tra i soggetti valutati, cruciali per comprendere la diversità delle risposte individuali.

In [19]:
summary(m2)

Linear mixed model fit by REML ['lmerMod']
Formula: slp_VAS ~ 1 + (1 | Speaker) + (1 | slpID)
   Data: slp_dat

REML criterion at convergence: 3544.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3281 -0.5813  0.0862  0.6611  2.5747 

Random effects:
 Groups   Name        Variance Std.Dev.
 slpID    (Intercept) 132.8    11.53   
 Speaker  (Intercept) 585.7    24.20   
 Residual             291.2    17.07   
Number of obs: 403, groups:  slpID, 21; Speaker, 20

Fixed effects:
            Estimate Std. Error t value
(Intercept)   61.882      6.028   10.27

Estraiamo le fonti di varianza:

In [20]:
vc_m2 <- as.data.frame(VarCorr(m2))
vc_m2

grp,var1,var2,vcov,sdcor
<chr>,<chr>,<chr>,<dbl>,<dbl>
slpID,(Intercept),NA,132.8257,11.52500
Speaker,(Intercept),NA,585.6568,24.20035
Residual,NA,NA,291.2401,17.06576


### Componenti di Varianza

- **Variazione dovuta a `slpID` (Intercept):** Indica la variabilità attribuibile alle differenze tra i soggetti.
- **Variazione dovuta a `Speaker` (Intercept):** Riflette la variabilità nelle valutazioni dovuta al bias dei giudici.
- **Residui (Residual):** Comprende l'errore di misurazione e altre fonti di variabilità non spiegate.

### Calcolo dell'ICC

Per calcolare l'ICC, dobbiamo distinguere tra le diverse versioni dell'ICC basate sulle definizioni generali. Tuttavia, il calcolo specifico dell'ICC può variare a seconda della struttura del modello e dell'interpretazione desiderata. Nella versione più semplice e comune, si considera la formula:

$$ 
\text{ICC} = \frac{\sigma^2_{\text{tra gruppo}}}{\sigma^2_{\text{tra gruppo}} + \sigma^2_{\text{errore}}} $$

Dove $ \sigma^2_{\text{tra gruppo}} $ è la varianza attribuita agli effetti casuali tra gruppi, e $ \sigma^2_{\text{errore}} $ è la varianza residua.

Nel modello precedente, ci sono due componenti di varianza tra gruppi (`slpID` e `Speaker`), quindi è possibile considerare la somma di queste due come la varianza totale tra gruppo. Così, l'ICC adjusted può essere calcolato come:

$$ 
\text{ICC}_{\text{adjusted}} = \frac{\sigma^2_{slpID} + \sigma^2_{Speaker}}{\sigma^2_{slpID} + \sigma^2_{Speaker} + \sigma^2_{Residual}} 
$$

Sostituendo i valori ottenuti:

$$ \text{ICC}_{\text{adjusted}} = \frac{132.8257 + 585.6568}{132.8257 + 585.6568 + 291.2401} $$

Questo calcolo assume che tutte le componenti di varianza contribuiscano al calcolo dell'ICC in modo equivalente e che l'ICC adjusted consideri la somma delle varianze tra gruppi (in questo caso, `slpID` e `Speaker`) rispetto alla varianza totale (inclusa la varianza residua).

Calcoliamo ora questo valore usando Python per ottenere il risultato esatto.

In [21]:
(vc_m2$vcov[1] + vc_m2$vcov[2]) / (vc_m2$vcov[1] + vc_m2$vcov[2] + vc_m2$vcov[3])

[1] 0.7115643

Questo valore riproduce quello trovato da `performance::icc()`:

In [22]:
performance::icc(m2)

ICC_adjusted,ICC_conditional,ICC_unadjusted
<dbl>,<dbl>,<dbl>
0.7115643,0.7115643,0.7115643


L'Intraclass Correlation Coefficient (ICC) di 0.7115643 offre una misura dell'affidabilità o della coerenza delle valutazioni all'interno dei gruppi definiti nel tuo modello (in questo caso, `slpID` e `Speaker`). 

- **Valore dell'ICC**: Il valore di 0.71 indica un livello relativamente alto di coerenza o omogeneità tra le misurazioni all'interno dei gruppi rispetto alla variabilità totale. In altre parole, una proporzione sostanziale della varianza totale nei dati è attribuibile alle differenze tra i gruppi piuttosto che alle variazioni casuali all'interno dei gruppi o agli errori di misurazione.

- **Affidabilità delle Misure**: Un ICC più vicino a 1 suggerisce che le misure sono molto affidabili, poiché indica che la maggior parte della varianza nei dati può essere attribuita alle differenze sistematiche tra i gruppi piuttosto che al rumore casuale o agli errori. Nel tuo caso, un ICC di circa 0.71 può essere interpretato come indicativo di un'alta affidabilità, suggerendo che le differenze tra i gruppi (ad esempio, tra diversi `Speaker` o differenti `slpID`) sono significative e consistenti.

- **Implicazioni per la Ricerca**: Per la ricerca, un ICC alto come questo implica che il disegno dello studio e la misurazione utilizzata sono adeguatamente sensibili per distinguere tra gli individui o le unità all'interno dei gruppi definiti. Questo è particolarmente rilevante quando si studiano gli effetti di interventi o trattamenti specifici su gruppi distinti o si valuta la consistenza delle risposte tra i membri di un gruppo.

- **Contesto e Benchmark**: L'interpretazione dell'ICC dipende dal contesto specifico e dal campo di studio. Alcuni campi potrebbero considerare un ICC di 0.71 come eccellente, mentre altri potrebbero considerarlo solo moderatamente buono. È importante confrontare questo valore con benchmark o standard specifici del campo di interesse.

- **Struttura del Modello e Dati**: L'interpretazione dovrebbe anche tenere conto della struttura specifica del modello e della natura dei dati. Diversi tipi di ICC possono essere calcolati a seconda degli obiettivi dello studio e della configurazione del modello misto, quindi è cruciale assicurarsi che l'ICC calcolato sia il più appropriato per il tuo specifico contesto di ricerca.

In sintesi, un ICC di 0.7115643 indica un'alta affidabilità nelle misure all'interno dei gruppi definiti nel tuo modello, suggerendo che le variazioni osservate sono significativamente influenzate dalle differenze tra i gruppi piuttosto che dalla varianza casuale o dall'errore di misurazione.

## Conclusioni

L'*Intraclass Correlation Coefficient* (ICC) può essere interpretato come "la proporzione di varianza spiegata dalla struttura di raggruppamento nella popolazione". Questa struttura di raggruppamento implica che le misurazioni siano organizzate in gruppi (ad esempio, i punteggi di un test in una scuola possono essere raggruppati per classe, se ci sono più classi e a ciascuna è stato somministrato lo stesso test), e l'ICC indica quanto fortemente le misurazioni nello stesso gruppo si assomiglino. Questo indice varia da 0, se il raggruppamento non trasmette alcuna informazione, a 1, se tutte le osservazioni in un gruppo sono identiche (Gelman e Hill, 2007, p. 258). In altre parole, l'ICC - a volte concettualizzato come ripetibilità della misurazione - "può anche essere interpretato come la correlazione attesa tra due unità estratte casualmente che si trovano nello stesso gruppo" (Hox 2010), sebbene questa definizione potrebbe non applicarsi a modelli misti con strutture di effetti casuali più complesse. L'ICC può aiutare a determinare se un modello misto è necessario: un ICC pari a zero (o molto vicino a zero) significa che le osservazioni all'interno dei cluster non sono più simili tra loro rispetto a osservazioni di cluster diversi, e quindi considerarlo come un fattore casuale potrebbe non essere necessario.

- *Alto ICC* significa che i dati sono altamente raggruppati, il che significa che i risultati dipendono fortemente dai gruppi a cui appartengono le osservazioni.
- *Basso ICC* significa che i dati non sono altamente (o per niente) raggruppati, il che significa che i risultati non dipendono molto (o per niente) dai gruppi a cui appartengono le osservazioni.

**Differenza con R^2**

Il coefficiente di determinazione R^2 quantifica la proporzione di varianza spiegata da un modello statistico, ma la sua definizione in modelli misti è complessa (da qui l'esistenza di diversi metodi per calcolare un'approssimazione). L'ICC è associato a R^2 poiché entrambi sono rapporti di componenti di varianza. Più precisamente, R^2 rappresenta la proporzione della varianza spiegata (del modello completo), mentre l'ICC è la proporzione della varianza spiegata attribuibile agli effetti casuali. 

**Calcolo**

L'ICC viene calcolato dividendo la varianza degli effetti casuali, σ^2_i, per la varianza totale, cioè la somma della varianza degli effetti casuali e della varianza residua, σ^2_ε.

**ICC aggiustato e non aggiustato**

La funzione `icc()` calcola un ICC aggiustato e uno non aggiustato, che tengono conto di tutte le fonti di incertezza (cioè di tutti gli effetti casuali). Mentre l'ICC aggiustato si riferisce solo agli effetti casuali, l'ICC non aggiustato considera anche le varianze degli effetti fissi, aggiungendo più precisamente la varianza degli effetti fissi al denominatore della formula per calcolare l'ICC (vedi Nakagawa et al. 2017). Tipicamente, l'ICC aggiustato è di interesse quando l'analisi degli effetti casuali è di interesse. `icc()` restituisce un valore ICC anche per strutture di effetti casuali più complesse, come modelli con pendenze casuali o design nidificato (più di due livelli) ed è applicabile per modelli con distribuzioni diverse dalla gaussiana. 

## Session Info

In [42]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS Sonoma 14.4

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] C

time zone: Europe/Rome
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lme4_1.1-35.1     Matrix_1.6-5      ggokabeito_0.1.0  viridis_0.6.5    
 [5] viridisLite_0.4.2 ggpubr_0.6.0      ggExtra_0.10.1    bayesplot_1.11.1 
 [9] gridExtra_2.3     patchwork_1.2.0   semTools_0.5-6    semPlot_1.1.6    
[13] lavaan_0.6-17     psych_2.4.1       scales_1.3.0      markdown_1.12    
[17] knitr_1.45        lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
[21] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.1      
[25] tibble_3.2